In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import seaborn as sns

In [3]:
reg_data = pd.read_csv('C:/Users/Владислав/Data science/github/Data/problem1-reg_data.csv', sep=';')
reg_data.head(3)

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3


In [4]:
auth_data = pd.read_csv('C:/Users/Владислав/Data science/github/Data/problem1-auth_data.csv', sep=';')
auth_data.head(3)

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2


In [10]:
auth_data.shape

(9601013, 2)

In [11]:
reg_data.isnull().sum()

reg_ts    0
uid       0
dtype: int64

In [12]:
auth_data.isnull().sum()

auth_ts    0
uid        0
dtype: int64

In [14]:
reg_data.duplicated().sum()

0

In [15]:
auth_data.duplicated().sum()

0

In [314]:
def calculate_RR(reg_data, auth_data, start_interval='1998-11-18', end_interval=str(date.today())):
    start_interval = datetime.strptime(start_interval, '%Y-%m-%d')
    end_interval = datetime.strptime(end_interval, '%Y-%m-%d')
    
    reg_data['reg_date'] = reg_data.reg_ts.apply(lambda x: datetime.fromtimestamp(x)).dt.date
    auth_data['auth_date'] = auth_data.auth_ts.apply(lambda x: datetime.fromtimestamp(x)).dt.date
    
    reg_data.reg_date = pd.to_datetime(reg_data.reg_date)
    auth_data.auth_date = pd.to_datetime(auth_data.auth_date)
    
    reg_data = reg_data[(reg_data.reg_date >= start_interval) & (reg_data.reg_date <= end_interval)]
    
    all_data = reg_data.merge(auth_data, on='uid', how='left')
    all_data['life_time'] = (all_data.auth_date-all_data.reg_date).dt.days
    
    pre_ret = all_data.groupby(['reg_date', 'life_time'], as_index=0).agg({'uid' : 'nunique'}). \
        rename(columns={'uid' : 'retention'}).merge(reg_data.groupby('reg_date').agg({'uid': 'count'}), on='reg_date')
    
    pre_ret['RR'] = pre_ret.retention / pre_ret.uid
    pre_ret = pre_ret.query('life_time < 31')
    RR_table = pre_ret.pivot(index='reg_date', columns='life_time', values='RR')
    return RR_table

In [315]:
result = calculate_RR(reg_data, auth_data, start_interval='2020-06-01', end_interval='2020-06-10')

In [322]:
result

life_time,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
reg_date,,,,,,,,,,,,,,,,,,,,,
2020-06-01,1.0,0.014674,0.052091,0.039618,0.046222,0.059428,0.073368,0.052825,0.052825,0.047689,...,0.038151,0.035950,0.036684,0.041820,0.035216,0.031548,0.033015,0.031548,0.032282,0.018342
2020-06-02,1.0,0.021245,0.036630,0.043223,0.056410,0.060073,0.068132,0.057875,0.036630,0.054945,...,0.043956,0.033700,0.043956,0.031502,0.031502,0.026374,0.027106,0.024176,0.032967,0.030037
2020-06-03,1.0,0.019020,0.040234,0.042429,0.033650,0.055596,0.076079,0.057059,0.046818,0.054865,...,0.035845,0.032919,0.039503,0.038771,0.038771,0.032187,0.029261,0.029993,0.039503,0.027067
2020-06-04,1.0,0.017531,0.039445,0.046749,0.047480,0.058437,0.062089,0.057706,0.048941,0.040175,...,0.039445,0.038714,0.032871,0.039445,0.031410,0.032871,0.031410,0.027757,0.035793,0.026297
2020-06-05,1.0,0.018950,0.035714,0.039359,0.052478,0.053207,0.070700,0.041545,0.045918,0.053207,...,0.046647,0.027697,0.033528,0.037901,0.026968,0.034985,0.039359,0.032799,0.034257,0.029155
2020-06-06,1.0,0.018923,0.037118,0.050218,0.052402,0.058224,0.069141,0.055313,0.056769,0.049491,...,0.049491,0.028384,0.037118,0.034207,0.032023,0.021106,0.040029,0.025473,0.032023,0.028384
2020-06-07,1.0,0.013081,0.039244,0.051599,0.049419,0.059593,0.054506,0.061773,0.049419,0.041424,...,0.040698,0.030523,0.025436,0.030523,0.027616,0.031977,0.026163,0.026890,0.026163,0.028343
2020-06-08,1.0,0.013062,0.031930,0.049347,0.048621,0.063861,0.071843,0.044993,0.054427,0.043541,...,0.031930,0.037736,0.041364,0.032656,0.037736,0.030479,0.031930,0.035559,0.031930,0.026851
2020-06-09,1.0,0.017379,0.036930,0.039826,0.050688,0.056481,0.068067,0.057205,0.041274,0.052136,...,0.039826,0.030413,0.030413,0.031137,0.035482,0.029689,0.031861,0.026068,0.035482,0.020999
